# ओपनबीएनबी एमसीपी सर्वर इंटीग्रेशन के साथ सेमांटिक केर्नेल

यह नोटबुक दिखाती है कि सेमांटिक केर्नेल का उपयोग असली ओपनबीएनबी एमसीपी सर्वर के साथ कैसे किया जा सकता है, ताकि एमसीपीस्टूडियोप्लगइन का उपयोग करके वास्तविक एयरबीएनबी आवास खोजे जा सकें। एलएलएम एक्सेस के लिए, यह Azure AI Foundry का उपयोग करता है। अपने पर्यावरण वेरिएबल्स सेटअप करने के लिए, आप [सेटअप पाठ](/00-course-setup/README.md) का अनुसरण कर सकते हैं।


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP प्लगइन कनेक्शन बनाना

हम MCPStdioPlugin का उपयोग करके [OpenBnB MCP सर्वर](https://github.com/openbnb-org/mcp-server-airbnb) से कनेक्ट करेंगे। यह सर्वर @openbnb/mcp-server-airbnb पैकेज के माध्यम से Airbnb खोज सुविधा प्रदान करता है।


## क्लाइंट बनाना

इस उदाहरण में, हम अपने LLM एक्सेस के लिए Azure AI Foundry का उपयोग करेंगे। सुनिश्चित करें कि आपके पर्यावरण वेरिएबल्स सही तरीके से सेट किए गए हैं।


## पर्यावरण कॉन्फ़िगरेशन

Azure OpenAI सेटिंग्स को कॉन्फ़िगर करें। सुनिश्चित करें कि आपके पास निम्नलिखित पर्यावरण वेरिएबल्स सेट हैं:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP इंटीग्रेशन को समझना

यह नोटबुक **वास्तविक OpenBnB MCP सर्वर** से जुड़ता है, जो असली Airbnb खोज सुविधा प्रदान करता है।

### यह कैसे काम करता है:

1. **MCPStdioPlugin**: MCP सर्वर के साथ मानक इनपुट/आउटपुट संचार का उपयोग करता है  
2. **वास्तविक NPM पैकेज**: `@openbnb/mcp-server-airbnb` को npx के माध्यम से डाउनलोड और चलाता है  
3. **लाइव डेटा**: Airbnb API से वास्तविक संपत्ति डेटा लौटाता है  
4. **फंक्शन डिस्कवरी**: एजेंट MCP सर्वर से उपलब्ध फंक्शन्स को स्वचालित रूप से खोजता है  

### उपलब्ध फंक्शन्स:

OpenBnB MCP सर्वर आमतौर पर निम्नलिखित प्रदान करता है:
- **search_listings** - स्थान और मानदंड के आधार पर Airbnb संपत्तियों की खोज करें  
- **get_listing_details** - विशिष्ट संपत्तियों की विस्तृत जानकारी प्राप्त करें  
- **check_availability** - विशिष्ट तिथियों के लिए उपलब्धता जांचें  
- **get_reviews** - संपत्तियों के लिए समीक्षाएं प्राप्त करें  
- **get_host_info** - संपत्ति के होस्ट की जानकारी प्राप्त करें  

### आवश्यकताएँ:

- आपके सिस्टम पर **Node.js** स्थापित होना चाहिए  
- MCP सर्वर पैकेज डाउनलोड करने के लिए **इंटरनेट कनेक्शन**  
- **NPX** उपलब्ध होना चाहिए (Node.js के साथ आता है)  

### कनेक्शन का परीक्षण:

आप MCP सर्वर को मैन्युअल रूप से चलाकर परीक्षण कर सकते हैं:
```bash
npx -y @openbnb/mcp-server-airbnb
```

यह OpenBnB MCP सर्वर को डाउनलोड और शुरू करेगा, जिससे Semantic Kernel वास्तविक Airbnb डेटा के लिए जुड़ता है।


## OpenBnB MCP सर्वर के साथ एजेंट चलाना

अब हम उस AI एजेंट को चलाएंगे जो OpenBnB MCP सर्वर से जुड़ता है ताकि स्टॉकहोम में 2 वयस्कों और 1 बच्चे के लिए वास्तविक Airbnb आवास खोज सके। खोज मानदंडों को बदलने के लिए `user_inputs` सूची को संशोधित करने में संकोच न करें।


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# सारांश
बधाई हो! आपने एक AI एजेंट सफलतापूर्वक बनाया है जो Model Context Protocol (MCP) का उपयोग करके वास्तविक दुनिया के आवास खोज के साथ एकीकृत होता है:

## उपयोग की गई तकनीकें:
- Semantic Kernel - Azure OpenAI के साथ बुद्धिमान एजेंट बनाने के लिए
- Azure AI Foundry - LLM क्षमताओं और चैट पूर्णता के लिए
- MCP (Model Context Protocol) - मानकीकृत टूल एकीकरण के लिए
- OpenBnB MCP Server - वास्तविक Airbnb खोज कार्यक्षमता के लिए
- Node.js/NPX - बाहरी MCP सर्वर चलाने के लिए

## आपने क्या सीखा:
- MCP एकीकरण: Semantic Kernel एजेंट को बाहरी MCP सर्वरों से जोड़ना
- वास्तविक समय डेटा एक्सेस: लाइव APIs के माध्यम से वास्तविक Airbnb संपत्तियों की खोज
- प्रोटोकॉल संचार: एजेंट और MCP सर्वर के बीच stdio संचार का उपयोग
- फ़ंक्शन खोज: MCP सर्वरों से उपलब्ध फ़ंक्शन्स को स्वचालित रूप से खोजना
- स्ट्रीमिंग प्रतिक्रियाएं: वास्तविक समय में फ़ंक्शन कॉल को कैप्चर और लॉग करना
- HTML रेंडरिंग: एजेंट प्रतिक्रियाओं को स्टाइल किए गए टेबल और इंटरैक्टिव डिस्प्ले के साथ प्रारूपित करना

## अगले कदम:
- अतिरिक्त MCP सर्वरों को एकीकृत करें (मौसम, उड़ानें, रेस्तरां)
- MCP और A2A प्रोटोकॉल को मिलाकर एक बहु-एजेंट प्रणाली बनाएं
- अपने स्वयं के डेटा स्रोतों के लिए कस्टम MCP सर्वर बनाएं
- सत्रों के बीच स्थायी बातचीत मेमोरी लागू करें
- MCP सर्वर ऑर्केस्ट्रेशन के साथ एजेंट को Azure Functions पर तैनात करें
- उपयोगकर्ता प्रमाणीकरण और बुकिंग क्षमताएं जोड़ें



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
